In [1]:
import os as os
import sys as sys
import re as re
import pandas as pd
import numpy as np
import json as json
import pickle
import urllib as urllib
import zlib as zlib
import base64 as base64
from requests import Request, Session
import requests
from numpy import trapz
import itertools
import zlib as zlib
import base64 as base64
import pickle
import pathlib
from tqdm import tqdm
from datetime import datetime
import time
import copy
from Bio import SeqIO

pd.options.display.max_columns = 100
pd.options.display.min_rows = None
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 100

DATA_PATH = pathlib.Path('/data1/home/adpatter/gene-to-phenotype-predictions/adpatter/data/')
DNA_PATH = DATA_PATH.joinpath('gene_symbol_dna_sequence.pkl')
PROTEIN_PATH = DATA_PATH.joinpath('gene_symbol_protein_sequences.pkl')
EFFECT_SIZE_PATH = DATA_PATH.joinpath('capstone_body_weight_Statistical_effect_size_analysis_genotype_early_adult_scaled_13022023_gene_symbol_harmonized.pkl')

In [2]:
df_dna = pd.read_pickle(DNA_PATH)
df_protein = pd.read_pickle(PROTEIN_PATH)
df_effect_size = pd.read_pickle(EFFECT_SIZE_PATH)

In [3]:
df_effect_size.head(2)

,procedure_name,parameter_name,allele_accession_id,gene_accession_id,gene_symbol,allele_symbol,zygosity,strain,est_f_ea,se_f_ea,p_f_ea,p_adj_f_ea,est_m_ea,se_m_ea,p_m_ea,p_adj_m_ea,est_type_ea,add_info_ea,wt_f_ea_count,wt_m_ea_count,mut_f_ea_count,mut_m_ea_count,total_number_of_mutants,gene_symbol_harmonized
0,body composition (dexa lean/fat),body weight,mgi:5694170,MGI:1913300,0610009B22Rik,0610009B22Rik<tm1b(EUCOMM)Hmgu>,homozygote,c57bl/6n,-0.56979,0.194647,0.003428,0.013711,-0.56979,0.194647,0.003428,0.00457,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes included"", ""multibatch_in_analysis"":""Data contains mul...",5034.0,5020.0,7.0,8.0,15.0,0610009B22Rik
1,body weight,body weight,not-released-0541dc0af7,MGI:1923511,0610040J01Rik,0610040J01Rik<em1(IMPC)Ccpcz>,homozygote,c57bl/6ncrl,-0.956256,0.558026,0.086929,0.115905,-0.956256,0.558026,0.086929,0.115905,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes included"", ""multibatch_in_analysis"":""Data contains mul...",5083.0,5198.0,8.0,8.0,16.0,0610040J01Rik


In [4]:
df_dna.head(2)

,Gene stable ID,Gene name,Sequence
0,ENSMUSG00000064336,mt-Tf,GTTAATGTAGCTTAATAACAAAGCAAAGCACTGAAAATGCTTAGATGGATAATTGTATCCCATAAACA
1,ENSMUSG00000064337,mt-Rnr1,AAAGGTTTGGTCCTGGCCTTATAATTAATTAGAGGTAAAATTACACATGCAAACCTCCATAGACCGGTGTAAAATCCCTTAAACATTTACTTAAAA...


In [5]:
df_protein.head(2)

,gene_symbol_harmonized,db,UniqueIdentifier,EntryName,ProteinName,OS,OX,GN,PE,SV,seq
0,Hsf1,tr,A0A075F5C6,A0A075F5C6_MOUSE,Heat shock factor protein 1,Mus musculus,10090,Hsf1,1,1,MDLAVGPGAAGPSNVPAFLTKLWTLVSDPDTDALICWSPSGNSFHVFDQGQFAKEVLPKYFKHNNMASFVRQLNMYGFRKVVHIEQGGLVKPERDD...
1,Auts2,sp,A0A087WPF7,AUTS2_MOUSE,Autism susceptibility gene 2 protein homolog,Mus musculus,10090,Auts2,1,2,MDGPTRGHGLRKKRRSRSQRDRERRSRAGLGTGAAGGIGAGRTRAPSLASSSGSDKEDNGKPPSSAPSRPRPPRRKRRESTSAEEDIIDGFAMTSF...


In [6]:
(~df_effect_size['gene_symbol_harmonized'].isin(df_dna['Gene name'])).sum()

18

In [7]:
(~df_effect_size['gene_symbol_harmonized'].isin(df_protein['GN'])).sum()

90

In [8]:
df = df_dna.copy()

df['gene_symbol'] = df['Gene name'].str.strip().str.lower()

df['gene_symbol'] = df['gene_symbol'].fillna('DNA_' + df['Gene stable ID'])

df['gene_sequence'] = df['Sequence'].copy()

df = df[['gene_symbol', 'gene_sequence']]

df = df.drop_duplicates()

df_dna_harmonized = df.copy()

df.shape

(57010, 2)

In [9]:
df = df_protein.copy()

df['gene_symbol'] = df['GN'].str.strip().str.lower()

df['gene_symbol'] = df['gene_symbol'].fillna('PROTEIN_' + df['UniqueIdentifier'])

df['protein_sequence'] = df['seq'].copy()

df = df[['gene_symbol', 'protein_sequence']]

df = df.drop_duplicates()

df_protein_harmonized = df.copy()

df.shape

(91635, 2)

In [10]:
df = df_dna_harmonized.merge(df_protein_harmonized, how='outer')

df = df.loc[df['gene_symbol'] == 'lepr']

df

,gene_symbol,gene_sequence,protein_sequence
105150,lepr,AGTGCCCCGCAGCGTCGCGCCCGCGCACGAGTGGTCGGCACCTTCTGAGCCAGCAGCATAGAGCCCAGGCATCCCTGCTTTACGCCACTCCGGGAC...,MMCQKFYVVLLHWEFLYVIAALNLAYPISPWKFKLFCGPPNTTDDSFLSPAGAPNNASALKGASEAIVEAKFNSSGIYVPELSKTVFHCCFGNEQG...
105151,lepr,AGTGCCCCGCAGCGTCGCGCCCGCGCACGAGTGGTCGGCACCTTCTGAGCCAGCAGCATAGAGCCCAGGCATCCCTGCTTTACGCCACTCCGGGAC...,MMCQKFYVVLLHWEFLYVIAALNLAYPISPWKFKLFCGPPNTTDDSFLSPAGAPNNASALKGASEAIVEAKFNSSGIYVPELSKTVFHCCFGNEQG...
105152,lepr,AGTGCCCCGCAGCGTCGCGCCCGCGCACGAGTGGTCGGCACCTTCTGAGCCAGCAGCATAGAGCCCAGGCATCCCTGCTTTACGCCACTCCGGGAC...,MMCQKFYVVLLHWEFLYVIAALNLAYPISPWKFKLFCGPPNTTDDSFLSPAGAPNNASALKGASEAIVEAKFNSSGIYVPELSKTVFHCCFGNEQG...
105153,lepr,AGTGCCCCGCAGCGTCGCGCCCGCGCACGAGTGGTCGGCACCTTCTGAGCCAGCAGCATAGAGCCCAGGCATCCCTGCTTTACGCCACTCCGGGAC...,MMCQKFYVVLLHWEFLYVIAALNLAYPISPWKFKLFCGPPNTTDDSFLSPAGAPNNASALKGASEAIVEAKFNSSGIYVPELSKTVFHCCFGNEQG...
105154,lepr,AGTGCCCCGCAGCGTCGCGCCCGCGCACGAGTGGTCGGCACCTTCTGAGCCAGCAGCATAGAGCCCAGGCATCCCTGCTTTACGCCACTCCGGGAC...,MMCQKFYVVLLHWEFLYVIAALNLAYPISPWKFKLFCGPPNTTDDSFLSPAGAPNNASALKGASEAIVEAKFNSSGIYVPELSKTVFHCCFGNEQG...
105155,lepr,AGTGCCCCGCAGCGTCGCGCCCGCGCACGAGTGGTCGGCACCTTCTGAGCCAGCAGCATAGAGCCCAGGCATCCCTGCTTTACGCCACTCCGGGAC...,MMCQKFYVVLLHWEFLYVIAALNLAYPISP
105156,lepr,AGTGCCCCGCAGCGTCGCGCCCGCGCACGAGTGGTCGGCACCTTCTGAGCCAGCAGCATAGAGCCCAGGCATCCCTGCTTTACGCCACTCCGGGAC...,MMCQKFYVVLLHWEFLYVIAALNLAYPISPWKFKLFCGPPNTTDDSFLSPAGAPNNASALKGASEAIVEAKFNSSGIYVPELSKTVFHCCFG
105157,lepr,AGTGCCCCGCAGCGTCGCGCCCGCGCACGAGTGGTCGGCACCTTCTGAGCCAGCAGCATAGAGCCCAGGCATCCCTGCTTTACGCCACTCCGGGAC...,MMCQKFYVVLLHWEFLYVIAALNLAYPISPWKFKLFCGPPNTTDDSFLSPAGAPNNASALKGASEAIVEAKFNSSGIYVPELSKTVFHCCFGNEQG...
105158,lepr,AGTGCCCCGCAGCGTCGCGCCCGCGCACGAGTGGTCGGCACCTTCTGAGCCAGCAGCATAGAGCCCAGGCATCCCTGCTTTACGCCACTCCGGGAC...,PMRGPEFWRKMDGDVTKKREMSPCFG


In [11]:
# df = df_gene_protein.copy()

# df.loc[df['Gene stable ID'].duplicated()]